# Time Period: 20140331-20140630

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

### X1: BE/ME Ratio in 2014 (same throughout the year)

In [3]:
AShareEODDerivativeIndicator = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/EODDerivativeIndicator.csv", header=0)
BEME_ratio_df_2014 = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Intermediate Files/BEME_ratio_df_2014.csv", header=0)
norm_BEME_ratio_list = list(BEME_ratio_df_2014['norm_BEME_ratio'])# 268 companies available total

### X2: Last Quarter Return: 2014 Q1

Duration: 20140102 - 20140331

return = [SP(20140331) - SP(20131231)]/SP(20131231)

sample pool: companies in BEME_ratio_df['WINDCODE']

In [4]:
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

In [5]:
start_close = [None]*len(BEME_ratio_df_2014['WINDCODE'])
end_close = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [18]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    company = BEME_ratio_df_2014['WINDCODE'][i]
    start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20131231]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20140331]['S_DQ_ADJCLOSE'])

In [20]:
com_qtr_return = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [21]:
# Calculate quarterly return
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

In [22]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [23]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2014-01-02']['last_price']) # index price on start day
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2014-03-31']['last_price'])  # index price on end day

In [24]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [25]:
print index_qtr_return

-0.0756507697276


In [26]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [27]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [28]:
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'diff_csi300_LAST': diff_return_com_csi300})

In [29]:
# Normalize Function
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [30]:
Weighted_Return['norm_diff_csi300_LAST'] = normalize_list(Weighted_Return['diff_csi300_LAST'])

In [32]:
Weighted_Return.to_csv("LAST_Weighted_Return_2014_Q2.csv")

In [33]:
###### X2: norm_diff_csi300_LAST

### 3. AShareEODDerivativeIndicator['FLOAT_A_SHR_TODAY']
total free-float shares per day * unadjusted close price

In [34]:
# Extract trading dates of certain period 
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

In [35]:
start_dateind = list(dates_trade).index(20140331) # index of start date
end_dateind = list(dates_trade).index(20140630) # index of end date

In [36]:
target_dates = dates_trade[start_dateind:end_dateind+1]

In [37]:
target_dates

array([20140331, 20140401, 20140402, 20140403, 20140404, 20140408,
       20140409, 20140410, 20140411, 20140414, 20140415, 20140416,
       20140417, 20140418, 20140421, 20140422, 20140423, 20140424,
       20140425, 20140428, 20140429, 20140430, 20140505, 20140506,
       20140507, 20140508, 20140509, 20140512, 20140513, 20140514,
       20140515, 20140516, 20140519, 20140520, 20140521, 20140522,
       20140523, 20140526, 20140527, 20140528, 20140529, 20140530,
       20140603, 20140604, 20140605, 20140606, 20140609, 20140610,
       20140611, 20140612, 20140613, 20140616, 20140617, 20140618,
       20140619, 20140620, 20140623, 20140624, 20140625, 20140626,
       20140627, 20140630])

In [38]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [39]:
FAST_mean_list = [find_FAST_mean(company) for company in BEME_ratio_df_2014['WINDCODE']]

In [40]:
FAST_mean_df = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'FAST_mean':FAST_mean_list})

In [41]:
FAST_mean_df['FAST_mean_norm'] = normalize_list(FAST_mean_df['FAST_mean'])

In [42]:
FAST_mean_df

,Company,FAST_mean,FAST_mean_norm
0,002475.SZ,2.246709e+06,-0.176382
1,002470.SZ,1.138795e+06,-0.273571
2,600036.SH,2.078233e+07,1.449617
3,002146.SZ,1.789176e+06,-0.216518
4,601988.SH,5.214953e+07,4.201238
5,000895.SZ,4.429710e+06,0.015117
6,600048.SH,5.402829e+06,0.100482
7,000718.SZ,6.283123e+05,-0.318352
8,002304.SZ,4.732042e+06,0.041639
9,600031.SH,4.054059e+06,-0.017836


In [43]:
FAST_mean_df.to_csv("FAST_mean_2014_Q2.csv") # save it as csv

## Y : Current Quarter Return

Duration: 20140331 - 20140630

sample pool: companies in BEME_ratio_df['WINDCODE']

In [44]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(BEME_ratio_df_2014['WINDCODE'])
end_close_c = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [46]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    company = BEME_ratio_df_2014['WINDCODE'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140331]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20140630]['S_DQ_ADJCLOSE'])

In [47]:
# create an empty list to store return for individual company for current qtr
com_qtr_return_CURRENT = [None]*len(BEME_ratio_df_2014['WINDCODE']) 

In [48]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [49]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-03-31']['last_price']) 
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-06-30']['last_price']) 

In [50]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [51]:
index_qtr_return_CURRENT

0.006494081255211828

In [52]:
diff_return_com_csi300_CURRENT = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [53]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

In [54]:
Y_2014_Q2 = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'Y':diff_return_com_csi300_CURRENT})

In [55]:
Y_2014_Q2.to_csv("Y_2014_Q2.csv")

In [56]:
TFM_2014_Q2 = pd.DataFrame({'X_1':norm_BEME_ratio_list,'X_2':Weighted_Return['norm_diff_csi300_LAST'], 'X_3': FAST_mean_df['FAST_mean_norm'], 'Y':diff_return_com_csi300_CURRENT})

In [58]:
TFM_2014_Q2.to_csv("TFM_2014_Q2.csv")

In [59]:
def outlier_index(list_name, std_num):
    mean = list_name.mean() # mean
    std = np.std(list_name, axis=0) # standard deviation
    
    upper_b = mean+std_num*std
    lower_b = mean-std_num*std
    
    ind_upper = [i for i in range(len(list_name)) if list_name[i] > upper_b]
    ind_lower = [i for i in range(len(list_name)) if list_name[i] < lower_b]
    outlier_index = ind_upper+ind_lower
    
    return outlier_index

In [60]:
outlier_all =outlier_index(TFM_2014_Q2['X_1'], 5) + outlier_index(TFM_2014_Q2['X_2'], 5)+ outlier_index(TFM_2014_Q2['X_3'], 5)+ outlier_index(TFM_2014_Q2['Y'], 5)

In [61]:
TFM_2014_Q2_NO = TFM_2014_Q2.drop(TFM_2014_Q2.index[outlier_all])

In [63]:
TFM_2014_Q2_NO[:10]

,X_1,X_2,X_3,Y
0,-0.943715,0.486337,-0.176382,0.272115
1,-0.663215,-0.412501,-0.273571,0.007317
2,0.831511,-0.381677,1.449617,0.036098
3,0.060847,1.278933,-0.216518,-0.224168
4,1.576604,0.050574,4.201238,0.054552
5,-0.981901,-0.716544,0.015117,-0.060494
6,1.036689,-0.278581,0.100482,0.010955
7,-0.199968,-0.198255,-0.318352,0.044800
8,-0.417362,1.129183,0.041639,0.064601
9,-0.118337,-0.559517,-0.017836,-0.100004


In [64]:
TFM_2014_Q2_NO.to_csv("TFM_2014_Q2_NO.csv")

# ************* Result DataFrame: TFM_2014_Q2_NO.csv *************